In [ ]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter
# Function to load the model and label encoder from disk and predict suitable crops for a given year and month
def predict_crops_from_saved_model(year, month, model, label_encoder, crop_to_human_category):
    # Load the weather data for the specified year and month
    webhdfs_url = "http://127.0.0.1:9870/webhdfs/v1/user/nourishnation/vap_future.csv?op=OPEN"
    response_vap = requests.get(webhdfs_url)
    data_vap = response_vap.text
    #print(data_vap)
    vap_data = pd.read_csv(StringIO(data_vap))
    webhdfs_url = "http://127.0.0.1:9870/webhdfs/v1/user/nourishnation/pre_future.csv?op=OPEN"
    response_pre = requests.get(webhdfs_url)
    data_pre = response_pre.text
    #print("\n\n", data_pre)
    pre_data = pd.read_csv(StringIO(data_pre))
    webhdfs_url = "http://127.0.0.1:9870/webhdfs/v1/user/nourishnation/temp_future.csv?op=OPEN"
    response_temp = requests.get(webhdfs_url)
    data_temp = response_temp.text
    #print("\n\n", data_temp)
    temp_data = pd.read_csv(StringIO(data_temp))
    # Filter data for the selected year
    vap_data_year = vap_data[vap_data['YEAR'] == year]
    pre_data_year = pre_data[pre_data['YEAR'] == year]
    temp_data_year = temp_data[temp_data['YEAR'] == year]
    # Extract weather data for the specified year and month
    selected_month_data = {
        'temperature': temp_data_year[month].values,
        'humidity': vap_data_year[month].values,
        'ph': np.round(np.random.uniform(5, 7, size=len(temp_data_year)), 6),
        'rainfall': pre_data_year[month].values
    }
    # Prepare the data for prediction
    selected_weather_data = pd.DataFrame(selected_month_data)
    # Make predictions using the loaded model
    predictions = model.predict(selected_weather_data)
    predicted_crops = label_encoder.inverse_transform(predictions)
    # Map each predicted crop to its human category
    predicted_crops_with_category = [(crop, crop_to_human_category.get(crop, 'Unknown category')) for crop in predicted_crops]
    return predicted_crops_with_category
 

def main():
    webhdfs_url = "http://127.0.0.1:9870/webhdfs/v1/user/nourishnation/crop_human.csv?op=OPEN"
    response_crop_human = requests.get(webhdfs_url)
    crop_data = pd.read_csv(StringIO(response_crop_human.text))
    crop_to_human_category = dict(zip(crop_data['label'], crop_data['human_category']))
    # Extracting all possible crops for completeness
    all_possible_crops = set(crop_data['label'].unique())
    # Load the model and label encoder from disk
    with open('/Users/roshinisampath/Crop-Recommendation/data/crop_prediction_model.pkl', 'rb') as file:
        model, label_encoder = pickle.load(file)
    available_months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
    print("Available months: " + ", ".join(available_months))
    year = int(input("Please enter the year for which you want to predict suitable crops: "))
    selected_month = input("Please enter the month: ").upper()
    # Validate user input
    if selected_month not in available_months:
        print("Invalid month. Please enter a valid month from the list.")
        return
    # Predict and create the priority list for the selected year and month
    predicted_crops_with_category = predict_crops_from_saved_model(year, selected_month, model, label_encoder, crop_to_human_category)
    # Create the priority list with human categories
    crop_counts = Counter([crop for crop, category in predicted_crops_with_category])
    priority_list = [crop for crop, count in crop_counts.most_common()]
    # Append non-predicted crops to the list
    non_predicted_crops = all_possible_crops - set(priority_list)
    complete_priority_list = priority_list + list(non_predicted_crops)
    print(f"Complete list of crops for {year}-{selected_month} ranked by suitability (with human categories):")
    for crop in complete_priority_list:
        human_category = crop_to_human_category.get(crop, 'Unknown category')
        print(f"{crop} (Human category: {human_category})")
 
# Call the main function
if __name__ == "__main__":
 
    main()